In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada-filtrados-500.csv', sep = '|')
df['filtrado_500']=df['filtrado_500'].astype(str)
df.head()

,acordao,areas,filtrado_500
0,297/2016-P,Responsabilidade,conta conta gerência instituto nacional seguro...
1,366/2016-P,Finanças Públicas,senado petróleo petróleo regime aduaneiro expo...
2,944/2016-P,Responsabilidade,tribunal conta plenário embargo declaração aco...
3,30/2016-P,Direito Processual,embargo declaração inss recorrente marco antôn...
4,55/2016-P,Pessoal,senac senac senac empresa senac giselle araújo...


In [2]:
df.shape

(10524, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Competência do TCU,Contrato Administrativo',
       'Competência do TCU,Contrato Administrativo,Responsabilidade',
       'Competência do TCU,Convênio',
       'Competência do TCU,Convênio,Direito Processual,Licitação',
       'Competência do TCU,Convênio,Gestão Administrativa',
       'Competência do TCU,Convênio,Responsabilidade',
       'Competência do TCU,Desestatização',
       'Competência do TCU,Direito Processual',
       'Competência do TCU,Direito Processual,Licitação',
       'Competência do TCU,Direito Processual,Pessoal',
       'Competência do TCU,Direito Processual,Responsabilidade',
       'Competência do TCU,Finanças Públicas',
       'Competência do TCU,Finanças Públicas,Licitação',
       'Competência do TCU,Finanças Públicas,Responsabilidade',
       'Competência do TCU,Gestão Administrativa',
       'Competência do TCU,Licitação',
       'Competência do TCU,Licitação,Pessoal',
       'Competência do TCU,Licitação,Responsabilidade',
   

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(10524, 91)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 35000
limite_texto = 500
dim_vetor = 100

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado_500'])

sequences = tokenizer.texts_to_sequences(df['filtrado_500'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 33339 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (10524, 500)


In [7]:
x.shape, y.shape

((10524, 500), (10524, 91))

In [8]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0314 19:45:14.122005 140509239658304 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0314 19:45:14.135898 140509239658304 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0314 19:45:14.137837 140509239658304 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0314 19:45:14.179296 140509239658304 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is d

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          3500000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              102402048 
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 91)                46683     
Total para